In [2]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

# Maryland

## VEST Documentation

Maryland
--------
Election results from Maryland State Board of Elections (https://elections.maryland.gov/elections/2016/election_data/index.html)  
Precinct shapefile from Maryland Department of Planning  

Early, Provisional, and Absentee votes are reported only at the county level. These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within a county.

G16PRERTru - Donald J. Trump (Republican Party)  
G16PREDCli - Hillary Clinton (Democratic Party)  
G16PRELJoh - Gary Johnson (Libertarian Party)  
G16PREGSte - Jill Stein (Green Party)  
G16PREOth - Write-in Votes  
  
G16USSRSze - Kathy Szeliga (Republican Party)  
G16USSDVan - Chris Van Hollen (Democratic Party)  
G16USSGFlo - Margaret Flowers (Green Party)  
G16USSOth - Write-in Votes  

G16H01RHar - Andy Harris (Republican Party)  
G16H01DWer - Joe Werner (Democratic Party)  
G16H01LBee - Matt Beers (Libertarian Party)  
G16H01Oth - Write-in Votes  

G16H02RMcD - Pat McDonough (Republican Party)  
G16H02DRup - C. A. Dutch Ruppersberger (Democratic Party)  
G16H02LKas - Kristin S. Kasprzak (Libertarian Party)  
G16H02Oth - Write-in Votes  
  
G16H03RPla - Mark Plaster (Republican Party)  
G16H03DSar - John Sarbanes (Democratic Party)  
G16H03GEze - Nnabu Eze (Green Party)  
G16H03Oth - Write-in Votes  

G16H04RMcD - George E. McDermott (Republican Party)  
G16H04DBro - Anthony G. Brown (Democratic Party)  
G16H04LKra - Benjamin Lee Krause (Libertarian Party)  
G16H04GCla - Kamesha T. Clark (Green Party)  
G16H04Oth - Write-in Votes  

G16H05RArn - Mark Arness (Republican Party)  
G16H05DHoy - Steny H. Hoyer (Democratic Party)  
G16H05LSum - Jason Summers (Libertarian Party)  
G16H05Oth - Write-in Votes  

G16H06RHoe - Amie Hoeber (Republican Party)  
G16H06DDel - John K. Delaney (Democratic Party)  
G16H06LHow - David L. Howser (Libertarian Party)  
G16H06GGlu - George Gluck (Green Party)  
G16H06Oth - Write-in Votes  

G16H07RVau - Corrogan R. Vaughn (Republican Party)  
G16H07DCum - Elijah Cummings (Democratic Party)  
G16H07GHoe - Myles B. Hoenig (Green Party)  
G16H07Oth - Write-in Votes  

G16H08RCox - Dan Cox (Republican Party)  
G16H08DRas - Jamie Raskin (Democratic Party)  
G16H08LWun - Jasen Wunder (Libertarian Party)  
G16H08GWal - Nancy Wallace (Green Party)  
G16H08Oth - Write-in Votes  

In [4]:
vest_md_16 = gp.read_file("./raw-from-source/VEST/md_2016/md_2016_w_ushouse.shp")

In [5]:
source_elections = pd.read_csv("./raw-from-source/Election_Results/All_By_Precinct_2016_General.csv")

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
print(source_elections.head())

   County  Election District  Election Precinct  Cong Legs   Candidate Name  \
0       1                  1                  0     6  01C  Donald J. Trump   
1       1                  1                  0     6  01C  Hillary Clinton   
2       1                  1                  0     6  01C     Gary Johnson   
3       1                  1                  0     6  01C       Jill Stein   
4       1                  1                  0     6  01C      Paij Boring   

  Party            Office Name Office District Winner Write-In?  \
0   REP  President - Vice Pres             NaN    NaN       NaN   
1   DEM  President - Vice Pres             NaN      Y       NaN   
2   LIB  President - Vice Pres             NaN    NaN       NaN   
3   GRN  President - Vice Pres             NaN    NaN       NaN   
4   REP  President - Vice Pres             NaN    NaN         Y   

   Election Night Votes  Election Night Votes Against  
0                   350                           NaN  
1         